In [18]:
import pandas as pd
import urllib.request

!apt install python3-tk ghostscript
!pip install camelot-py[cv]

from lxml import html
import requests

# OBTÉM TODOS OS LINKS DISPINÍVEIS NA PÁGINA E EXTRAI O PRIMEIRO LINK QUE CONTÉM O PDF MAIS RECENTE



page = requests.get('http://www.saude.pr.gov.br/modules/conteudo/conteudo.php?conteudo=3507')
webpage = html.fromstring(page.content)

# webpage.xpath('//a/@href')
links = webpage.xpath('//a/@href')


for link in links:
    if 'http://www.saude.pr.gov.br/arquivos/File/CORONA' in link:
        link_pdf = link
        break

print(link_pdf)

# FAZ A CONVERSÃO DE PDF PARA DATAFRAME E POSTERIORMENTE SALVA COMO CSV

import camelot

url = link_pdf
req = urllib.request.urlretrieve(url, r'covid19_PR.pdf')
tables = camelot.read_pdf(r'covid19_PR.pdf', pages='1,2') # editar o numero de páginas caso de erro
# tables.export(r'covid19_PR.csv', f='csv', compress=False) # json, excel, html, sqlite
df = pd.concat([tables[1].df, tables[2].df]) 
df = df.iloc[ 1:df.shape[0]-1 , 1:df.shape[1]]
df.to_csv('covid19_PR.csv')
df = pd.read_csv(r'covid19_PR.csv')
df = df.rename(columns={'1':'MUNICÍPIO', '2':'CONFIRMADOS', '3':'DESCARTADOS', '4':'SUSPEITOS', '5':'TOTAL'})
df = df.sort_values(['MUNICÍPIO'], ascending=1)
# df = df.reset_index(drop=True)
df = df.iloc[ : , 1:df.shape[1]]
df = df.reset_index(drop=True)
df.to_csv('covid19_PR.csv', index=False)
df


Reading package lists... Done
Building dependency tree       
Reading state information... Done
ghostscript is already the newest version (9.26~dfsg+0-0ubuntu0.18.04.12).
python3-tk is already the newest version (3.6.9-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
http://www.saude.pr.gov.br/arquivos/File/CORONA_21032020.pdf


,MUNICÍPIO,CONFIRMADOS,DESCARTADOS,SUSPEITOS,TOTAL
0,Almirante Tamandaré,NaN,NaN,1.0,1
1,Alto Paraná,NaN,3.0,NaN,3
2,Altônia,NaN,NaN,1.0,1
3,Alvorada do Sul,NaN,NaN,1.0,1
4,Ampére,NaN,NaN,1.0,1
...,...,...,...,...,...
59,Telêmaco Borba,NaN,NaN,2.0,2
60,Toledo,NaN,NaN,5.0,5
61,Tomazina,NaN,NaN,1.0,1
62,Turvo,NaN,1.0,NaN,1
